In [15]:
def WSI_CP(root, des, CPI):
    import glob,os,imagecodecs,tifffile as tiff,numpy as np
    from patchify import patchify 
    os.chdir(des)
    if os.path.isfile(des+'Dirty_file.txt') is False:
        Dirty_files = open('Dirty_file.txt','w')
    else:
        Dirty_files = open('Dirty_file.txt','a')



    des = des+'WSI_CP'
    if os.path.isdir(des) is False:
        os.makedirs(des)
    else:
        EP = os.listdir(des)
        EP = list(set([i.split('_')[0] for i in EP]))
        

    CPI_filenames = os.listdir(CPI)

    for file in glob.glob(root+'*.tif'):
        
        file = file.replace("\\","/")
        filename = file[len(root):-4]
        to_read_file = ''
        if filename in EP:
            continue

        try:
            img = tiff.imread(file)
            patch_id = 0
        except:
            Dirty_files.write(f'{filename}')
        else:     
            for i in range(len(CPI_filenames)):
                if filename == CPI_filenames[i].split('_')[0]:
                    to_read_file = CPI_filenames[i]
                    CPI_filenames.pop(i)
                    break

            CP = []
            with open(CPI+to_read_file) as file:
                for i in file.readlines():
                    CP.append(int(i))

            patches = patchify(img, (256, 256,3), step=256)
            x, y, z = patches.shape[0], patches.shape[1], patches.shape[2]
            patch = []

            for i in range(x):
                for j in range(y):
                    for k in range(z):
                        if patch_id in CP:
                            patch.append(patches[i][j][k])
                        patch_id += 1
            


            for i,file in zip(CP,patch):
                tiff.imwrite(des+'/{}_CP_{}.jpg'.format(filename,i), file, photometric='rgb')
            
            if not CPI_filenames:
                break
            
            del img, CP,min_CP,max_CP, filename,patches,patch,x,y,z,patch_id,to_read_file

    Dirty_files.close()
        
    # !zip -r '/content/patches' '/content/patches'
    # files.download('{}.zip'.format(filename))

In [11]:
root = 'H:/Azib/Lung_cancer/LCDC/Train/'
des = 'H:/Azib/Lung_cancer/LCDC/'
CPI = 'H:/Azib/Lung_cancer/LCDC/cancer_patch_id/'

In [14]:
WSI_CP(root,des,CPI)

PermissionError: [Errno 13] Permission denied: 'H:/Azib/Lung_cancer/LCDC/cancer_patch_id/'